<a href="https://colab.research.google.com/github/steelpipe75/kagglebook-for-colab/blob/master/ch04/ch04-01-introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

In [2]:
import importlib
import sys
import subprocess

# Google Colab 上で実行しているかどうかを判断するフラグ
ON_COLAB = "google.colab" in sys.modules
print(f"ON_COLAB: {ON_COLAB}")

if ON_COLAB:
    USE_GIT = True # Gitを使う
    # USE_GIT = False # Gitを使わない

    print(f"USE_GIT: {USE_GIT}")
    if USE_GIT:
        !git clone https://github.com/ghmagazine/kagglebook.git
    else:
        # Google Drive にマウントする
        drive = importlib.import_module("google.colab.drive")
        drive.mount("/content/drive/")

        import os
        colab_dir = "/content/drive/MyDrive/kagglebook/" # データ置き場

ON_COLAB: True
USE_GIT: True
Cloning into 'kagglebook'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 95 (delta 19), reused 87 (delta 18), pack-reused 1 (from 1)
Receiving objects: 100% (95/95), 3.13 MiB | 6.64 MiB/s, done.
Resolving deltas: 100% (19/19), done.


train_xは学習データ、train_yは目的変数、test_xはテストデータ
pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

In [3]:
if ON_COLAB:
    if USE_GIT:
        train = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed.csv')
    else:
        train = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/train_preprocessed.csv'))
else:
    train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
if ON_COLAB:
    if USE_GIT:
        test_x = pd.read_csv('/content/kagglebook/input/sample-data/test_preprocessed.csv')
    else:
        test_x = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/test_preprocessed.csv'))
else:
    test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

In [4]:
import xgboost as xgb

In [5]:
# コードの動作を確認するためのモデル
class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y):
        params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        self.model = xgb.train(params, dtrain, num_round)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

In [6]:
# -----------------------------------
# モデルの学習と予測
# -----------------------------------
# モデルのハイパーパラメータを指定する
params = {'param1': 10, 'param2': 100}

Modelクラスを定義しているものとする
Modelクラスは、fitで学習し、predictで予測値の確率を出力する

In [7]:
# モデルを定義する
model = Model(params)

In [8]:
# 学習データに対してモデルを学習させる
model.fit(train_x, train_y)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:42:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "param1", "param2", "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [9]:
# テストデータに対して予測結果を出力する
pred = model.predict(test_x)

In [10]:
# -----------------------------------
# バリデーション
# -----------------------------------
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

In [11]:
# 学習データ・バリデーションデータを分けるためのインデックスを作成する
# 学習データを4つに分割し、うち1つをバリデーションデータとする
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]

In [12]:
# 学習データを学習データとバリデーションデータに分ける
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [13]:
# モデルを定義する
model = Model(params)

In [14]:
# 学習データに対してモデルを学習させる
# モデルによっては、バリデーションデータを同時に与えてスコアをモニタリングすることができる
model.fit(tr_x, tr_y)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:42:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "param1", "param2", "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [15]:
# バリデーションデータに対して予測し、評価を行う
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

logloss: 0.2949


In [16]:
# -----------------------------------
# クロスバリデーション
# -----------------------------------
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

In [17]:
# 学習データを4つに分け、うち1つをバリデーションデータとする
# どれをバリデーションデータとするかを変えて学習・評価を4回行う
scores = []
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    model = Model(params)
    model.fit(tr_x, tr_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    scores.append(score)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:42:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "param1", "param2", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:43:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "param1", "param2", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:43:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "param1", "param2", "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:43:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "param1", "param2", "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [18]:
# クロスバリデーションの平均のスコアを出力する
print(f'logloss: {np.mean(scores):.4f}')

logloss: 0.2912
